# TASK 11.3 (Changed)

W zestawieniu uwzglądniono kilka zestawów danych:
1. Współczynnik dzietności ('total_fertility_rate.csv'), liczba żywych urodzeń ('number_of_live_birth_by_sex.csv'), liczba aborcji ('number_of_abortion.csv') oraz liczba aborcji na 1000 żywych urodzeń ('abortion_per_1000_live_birth.csv') pobrane ze strony: 'https://gateway.euro.who.int/en/'
2. Objaśnienie symboli krajów zgodnie z normami ze strony: 'https://www.panstwaswiata.pl/lista-kodow-panstw-iso-3166-1/'
3. Listę krajów unii europejskiej (EU) ze strony: 'https://stat.gov.pl/badania-statystyczne/sprawozdawczosc/intrastat/wykaz-panstw-czlonkowskich-unii-europejskiej/'

Na podstawie powyższych danych wykonane będą następujące ćwiczenia:
1. Do listy krajów EU dodany będzie skrót trzyliterowy zgodnie z tabelą objaśnień symboli.
2. Ze wszystkich zestawień pobrancych z 'https://gateway.euro.who.int/en/' wybrane będą tylko kraje należące do EU i lata 2000 do 2020.
3. Przedstawione będą zestawienia dzietności, ilości żywych urodzeń i ilości aborcji dla każdego kraju EU jako średnia z tych 20 ostatnich lat.
4. Dla wybranych 3 krajów zaprezentowane będą statystyki dzietności, ilości żywych urodzeń i ilości aborcji dla każdego roku osobno.
5. Analiza dzietności w krajach EU na przestrzeni 2000-2020.


In [2]:
# import package
import numpy as np
import pandas as pd
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
# init_notebook_mode - to show plotly in Notebook's output
init_notebook_mode(connected = True)
import matplotlib.pyplot as plt
import plotly.offline as pyo
import plotly.graph_objs as go
from plotly import subplots
import plotly.figure_factory as ff

## Prepared data

In [3]:
fertility = pd.read_csv('total_fertility_rate.csv', sep = ',')
live_birth = pd.read_csv('number_of_live_birth_by_sex.csv')
abortion = pd.read_csv('number_of_abortion.csv')
abortion_per_1000 = pd.read_csv('abortion_per_1000_live_birth.csv')
print("FERILITY")
display(fertility.head())
print("LIVE BIRTH")
display(live_birth.head())
print("ABORTION")
display(abortion.head())
print("ABORTION PER 1000 LIVE BIRTH")
display(abortion_per_1000.head())

FERILITY


,COUNTRY,COUNTRY_GRP,SEX,YEAR,VALUE
0,ALB,NaN,ALL,1980.0,3.60
1,ALB,NaN,ALL,1981.0,3.44
2,ALB,NaN,ALL,1982.0,3.56
3,ALB,NaN,ALL,1983.0,3.25
4,ALB,NaN,ALL,1984.0,3.38


LIVE BIRTH


,COUNTRY,COUNTRY_GRP,SEX,YEAR,VALUE
0,ALB,NaN,ALL,1970.0,73000.0
1,ALB,NaN,ALL,1971.0,74000.0
2,ALB,NaN,ALL,1972.0,75000.0
3,ALB,NaN,ALL,1973.0,76000.0
4,ALB,NaN,ALL,1974.0,77000.0


ABORTION


,COUNTRY,COUNTRY_GRP,SEX,YEAR,VALUE
0,ALB,NaN,ALL,1980.0,15912.0
1,ALB,NaN,ALL,1981.0,15693.0
2,ALB,NaN,ALL,1982.0,16289.0
3,ALB,NaN,ALL,1983.0,17756.0
4,ALB,NaN,ALL,1984.0,18908.0


ABORTION PER 1000 LIVE BIRTH


,COUNTRY,COUNTRY_GRP,SEX,YEAR,VALUE
0,ALB,NaN,ALL,1980.0,225.13
1,ALB,NaN,ALL,1981.0,193.74
2,ALB,NaN,ALL,1982.0,198.65
3,ALB,NaN,ALL,1983.0,240.60
4,ALB,NaN,ALL,1984.0,238.81


In [4]:
# take only value with not null country, change SEX to All if Null, drop 'COUNTRY_GRP'
fertility_without_null_countries = fertility[fertility['COUNTRY'].notnull()]
fertility_without_null_countries['SEX'].fillna('ALL', inplace = True)
fertility_without_null_countries.drop(columns = 'COUNTRY_GRP', inplace = True)

live_birth_without_null_countries = live_birth[live_birth['COUNTRY'].notnull()]
live_birth_without_null_countries['SEX'].fillna('ALL', inplace = True)
live_birth_without_null_countries.drop(columns = 'COUNTRY_GRP', inplace = True)

abortion_without_null_countries = abortion[abortion['COUNTRY'].notnull()]
abortion_without_null_countries['SEX'].fillna('ALL', inplace = True)
abortion_without_null_countries.drop(columns = 'COUNTRY_GRP', inplace = True)

abortion_per_1000_without_null_countries = abortion_per_1000[abortion_per_1000['COUNTRY'].notnull()]
abortion_per_1000_without_null_countries['SEX'].fillna('ALL', inplace = True)
abortion_per_1000_without_null_countries.drop(columns = 'COUNTRY_GRP', inplace = True)

# join all tables
# join fertility with live_birth
data = pd.merge(fertility_without_null_countries, live_birth_without_null_countries, on = ['YEAR', 'COUNTRY', 'SEX'], how = 'outer')
data.rename(columns = {'VALUE_x' : 'FERITILITY_RANGE', 'VALUE_y' : 'LIVE_BIRTH'}, inplace = True)

# join data with abortion
data = pd.merge(data, abortion_without_null_countries, on = ['YEAR', 'COUNTRY', 'SEX'], how = 'outer')
data.rename(columns = {'VALUE' : 'ABORTION'}, inplace = True)

#join data with abortion_per_1000
data = pd.merge(data, abortion_per_1000_without_null_countries, on = ['YEAR', 'COUNTRY', 'SEX'], how = 'outer')
data.rename(columns = {'VALUE' : 'ABORTION_PER_1000'}, inplace = True)

data.head()

,COUNTRY,SEX,YEAR,FERITILITY_RANGE,LIVE_BIRTH,ABORTION,ABORTION_PER_1000
0,ALB,ALL,1980.0,3.60,70680.0,15912.0,225.13
1,ALB,ALL,1981.0,3.44,81000.0,15693.0,193.74
2,ALB,ALL,1982.0,3.56,82000.0,16289.0,198.65
3,ALB,ALL,1983.0,3.25,73800.0,17756.0,240.60
4,ALB,ALL,1984.0,3.38,79177.0,18908.0,238.81


In [5]:
# name of country acording to short name
url_countries = 'https://www.panstwaswiata.pl/lista-kodow-panstw-iso-3166-1/'
countries = pd.read_html(url_countries, header = 0)[0]
countries.head()

,polska nazwa,angielska nazwa,kod alfa-2,kod alfa-3,kod numeryczny,kody ISO 3166-2
0,Afganistan,Afghanistan,AF,AFG,4,ISO 3166-2:AF
1,Albania,Albania,AL,ALB,8,ISO 3166-2:AL
2,Algieria,Algeria,DZ,DZA,12,ISO 3166-2:DZ
3,Andora,Andorra,AD,AND,20,ISO 3166-2:AD
4,Angola,Angola,AO,AGO,24,ISO 3166-2:AO


In [6]:
url_EU = 'https://stat.gov.pl/badania-statystyczne/sprawozdawczosc/intrastat/wykaz-panstw-czlonkowskich-unii-europejskiej/'
countries_EU = pd.read_html(url_EU, header = 0)[0]
countries_EU

,SYMBOL ISO 3166,POLSKA NAZWA,ANGIELSKA NAZWA
0,AT,Austria,Austria
1,BE,Belgia,Belgium
2,BG,Bułgaria,Bulgaria
3,HR,Chorwacja,Croatia
4,CY,Cypr,Cyprus
5,CZ,Czechy,Czechia
6,DK,Dania,Denmark
7,EE,Estonia,Estonia
8,FI,Finlandia,Finland
9,FR,Francja,France


## TASK 1
Do listy krajów EU dodany będzie skrót trzyliterowy zgodnie z tabelą objaśnień symboli.

In [7]:
countries_EU_symbols = countries_EU.merge(countries, how='inner', left_on='SYMBOL ISO 3166', right_on='kod alfa-2')
countries_EU_symbols = countries_EU_symbols[['SYMBOL ISO 3166', 'ANGIELSKA NAZWA', 'kod alfa-3']]
countries_EU_symbols.rename(columns = {'SYMBOL ISO 3166' : 'SYMBOL', 'kod alfa-3' : 'CODE', 'ANGIELSKA NAZWA' : 'NAME'}, inplace = True)
countries_EU_symbols

,SYMBOL,NAME,CODE
0,AT,Austria,AUT
1,BE,Belgium,BEL
2,BG,Bulgaria,BGR
3,HR,Croatia,HRV
4,CY,Cyprus,CYP
5,CZ,Czechia,CZE
6,DK,Denmark,DNK
7,EE,Estonia,EST
8,FI,Finland,FIN
9,FR,France,FRA


## TASK 2
Ze wszystkich zestawień pobrancych z 'https://gateway.euro.who.int/en/' wybrane będą tylko kraje należące do EU i lata 2000 do 2020.

In [8]:
EU_countries_code = countries_EU_symbols['CODE'].unique()
print("Country group in dataset: ")
display(fertility['COUNTRY_GRP'].unique())
print("Country code (three letters) list: ")
display(EU_countries_code)

# wybranie z fertility, live_birth, abortion, abortion_per_1000 krajów z EU
def check_if_EU_country(row, list_of_countries):
    if row['COUNTRY'] is None:
        row['EU_COUNTRY'] = 0
    if row['COUNTRY'] in list_of_countries:
        row['EU_COUNTRY'] = 1
    else:
        row['EU_COUNTRY'] = 0
    return row

Country group in dataset: 


array([nan, 'CARINFONET', 'CIS', 'EU_AFTER_MAY2004', 'EU_BEFORE_MAY2004',
       'EU_MEMBERS', 'NORDIC', 'SEEHN', 'SMALL', 'WHO_EURO',
       '2022.08.31 14:50:27 GMT', 'Total fertility rate',
       'https://dw.euro.who.int/api/v3/measures/HFA_25?lang=En',
       'https://www.who.int/about/policies/publishing/copyright'],
      dtype=object)

Country code (three letters) list: 


array(['AUT', 'BEL', 'BGR', 'HRV', 'CYP', 'CZE', 'DNK', 'EST', 'FIN',
       'FRA', 'GRC', 'ESP', 'IRL', 'LTU', 'LUX', 'LVA', 'MLT', 'NLD',
       'DEU', 'POL', 'PRT', 'ROU', 'SVK', 'SVN', 'SWE', 'HUN', 'ITA'],
      dtype=object)

In [9]:
data = data.apply(lambda row: check_if_EU_country(row, EU_countries_code), axis = 1)
print("Add colum to define if country is EU member or not")
display(data.head())

# only EU contries data
data_EU = data[data['EU_COUNTRY'] == 1]
data_EU.drop(columns = {'EU_COUNTRY'}, inplace = True)
print("Only EU member caountries")
display(data_EU.head())

# between 2000 and 2020
data_EU_2000_2020 = data_EU[((data_EU['YEAR'] >= 2000) & (data['YEAR'] <= 2020))]
print('Only EU member caountries between 2000 and 2020')
display(data_EU_2000_2020.head())

Add colum to define if country is EU member or not


,COUNTRY,SEX,YEAR,FERITILITY_RANGE,LIVE_BIRTH,ABORTION,ABORTION_PER_1000,EU_COUNTRY
0,ALB,ALL,1980.0,3.60,70680.0,15912.0,225.13,0
1,ALB,ALL,1981.0,3.44,81000.0,15693.0,193.74,0
2,ALB,ALL,1982.0,3.56,82000.0,16289.0,198.65,0
3,ALB,ALL,1983.0,3.25,73800.0,17756.0,240.60,0
4,ALB,ALL,1984.0,3.38,79177.0,18908.0,238.81,0


Only EU member caountries


,COUNTRY,SEX,YEAR,FERITILITY_RANGE,LIVE_BIRTH,ABORTION,ABORTION_PER_1000
104,AUT,ALL,1980.0,1.60,90872.0,NaN,NaN
105,AUT,ALL,1981.0,1.67,93942.0,NaN,NaN
106,AUT,ALL,1982.0,1.66,94840.0,NaN,NaN
107,AUT,ALL,1983.0,1.56,90118.0,NaN,NaN
108,AUT,ALL,1984.0,1.52,89234.0,NaN,NaN


Only EU member caountries between 2000 and 2020


/var/folders/dl/n6r7g5t96w30wn8vg3gb44640000gn/T/ipykernel_1732/1921571368.py:12: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,COUNTRY,SEX,YEAR,FERITILITY_RANGE,LIVE_BIRTH,ABORTION,ABORTION_PER_1000
124,AUT,ALL,2000.0,1.36,78268.0,NaN,NaN
125,AUT,ALL,2001.0,1.33,75458.0,NaN,NaN
126,AUT,ALL,2002.0,1.40,78399.0,NaN,NaN
127,AUT,ALL,2003.0,1.38,76944.0,NaN,NaN
128,AUT,ALL,2004.0,1.42,78968.0,NaN,NaN


## TASK 3 
Przedstawione będą zestawienia dzietności, ilości żywych urodzeń i ilości aborcji dla każdego kraju EU jako średnia z tych 20 ostatnich lat.

In [10]:
data_by_country = data_EU_2000_2020.pivot_table(index = {'COUNTRY'}, aggfunc = 'mean')
data_by_country.drop(columns = 'YEAR', inplace = True)
data_by_country.reset_index(inplace = True)
data_by_country.head()

,COUNTRY,ABORTION,ABORTION_PER_1000,FERITILITY_RANGE,LIVE_BIRTH
0,AUT,NaN,NaN,1.429524,80243.857143
1,BEL,17585.700000,144.7515,1.730000,120938.333333
2,BGR,35220.380952,514.6465,1.439524,69239.333333
3,CYP,NaN,NaN,1.426190,9049.190476
4,CZE,23947.904762,231.1500,1.442381,107222.095238


In [11]:
# prepare plot
domain = data_by_country['COUNTRY']
fig = go.Figure()
fig = subplots.make_subplots(rows = 3, cols = 1, subplot_titles=['FERITILITY RANGE','LIVE BIRTH AND ABORTION', 'ABORTION PER 1000 LIVE BIRTH'], shared_xaxes = True)
fig.add_trace(go.Bar(x = domain, y = data_by_country['FERITILITY_RANGE'], name='FERITILITY RANGE'), 1, 1)
fig.add_trace(go.Bar(x = domain, y = data_by_country['LIVE_BIRTH'], name = 'LIVE BIRTH'), 2, 1)
fig.add_trace(go.Bar(x = domain, y = data_by_country['ABORTION'], name = 'ABORTION'), 2, 1)
fig.add_trace(go.Bar(x = domain, y = data_by_country['ABORTION_PER_1000'], name = 'ABORTION PER 1000 LIVE BIRTH'), 3, 1)
fig.update_layout(title = 'Zestawienie danych 2000-2020')
iplot(fig)

## TASK 4
Dla wybranych 3 krajów zaprezentowane będą statystyki dzietności, ilości żywych urodzeń i ilości aborcji dla każdego roku osobno.

In [12]:
# wybrane kraje Polska, Niemcy i Słowacja
poland = data_EU_2000_2020[data_EU_2000_2020['COUNTRY'] == 'POL']
germany = data_EU_2000_2020[data_EU_2000_2020['COUNTRY'] == 'DEU']
slovakia = data_EU_2000_2020[data_EU_2000_2020['COUNTRY'] == 'SVK']
print("POLSKA")
display(poland.head())
print("NIEMCY")
display(germany.head())
print("SŁOWACJA")
display(slovakia.head())

POLSKA


,COUNTRY,SEX,YEAR,FERITILITY_RANGE,LIVE_BIRTH,ABORTION,ABORTION_PER_1000
1567,POL,ALL,2000.0,1.37,378348.0,138.0,0.36
1568,POL,ALL,2001.0,1.29,368205.0,124.0,0.34
1569,POL,ALL,2002.0,1.24,353765.0,159.0,0.45
1570,POL,ALL,2003.0,1.22,351072.0,174.0,0.50
1571,POL,ALL,2004.0,1.23,356131.0,193.0,0.54


NIEMCY


,COUNTRY,SEX,YEAR,FERITILITY_RANGE,LIVE_BIRTH,ABORTION,ABORTION_PER_1000
502,DEU,ALL,2000.0,1.38,766969.0,134609.0,175.51
503,DEU,ALL,2001.0,1.35,734475.0,134964.0,183.76
504,DEU,ALL,2002.0,1.34,719250.0,130387.0,181.28
505,DEU,ALL,2003.0,1.34,706721.0,128030.0,181.16
506,DEU,ALL,2004.0,1.36,705622.0,129650.0,183.74


SŁOWACJA


,COUNTRY,SEX,YEAR,FERITILITY_RANGE,LIVE_BIRTH,ABORTION,ABORTION_PER_1000
1776,SVK,ALL,2000.0,1.29,55151.0,16580.0,300.63
1777,SVK,ALL,2001.0,1.20,51136.0,15899.0,310.92
1778,SVK,ALL,2002.0,1.18,50841.0,15301.0,300.96
1779,SVK,ALL,2003.0,1.20,51713.0,14159.0,273.80
1780,SVK,ALL,2004.0,1.24,53747.0,13146.0,244.59


In [13]:
# prepare plot
domain = data_EU_2000_2020['YEAR']
fig = go.Figure()
fig = subplots.make_subplots(rows = 4, cols = 1, subplot_titles=['FERITILITY RANGE','LIVE BIRTH', 'ABORTION', 'ABORTION PER 1000 LIVE BIRTH'])
fig.add_trace(go.Bar(x = domain, y = poland['FERITILITY_RANGE'], name = 'POLAND', legendgroup = 'FERITILITY'), 1, 1)
fig.add_trace(go.Bar(x = domain, y = germany['FERITILITY_RANGE'], name = 'GERMANY', legendgroup = 'FERITILITY'), 1, 1)
fig.add_trace(go.Bar(x = domain, y = slovakia['FERITILITY_RANGE'], name = 'SLOVAKIA', legendgroup = 'FERITILITY'), 1, 1)
fig.add_trace(go.Bar(x = domain, y = poland['LIVE_BIRTH'], name = 'POLAND', legendgroup = 'LIVE BIRTH'), 2, 1)
fig.add_trace(go.Bar(x = domain, y = germany['LIVE_BIRTH'], name = 'GERMANY', legendgroup = 'LIVE BIRTH'), 2, 1)
fig.add_trace(go.Bar(x = domain, y = slovakia['LIVE_BIRTH'], name = 'SLOVAKIA', legendgroup = 'LIVE BIRTH'), 2, 1)
fig.add_trace(go.Bar(x = domain, y = poland['ABORTION'], name = 'POLAND', legendgroup = 'ABORTION'), 3, 1)
fig.add_trace(go.Bar(x = domain, y = germany['ABORTION'], name = 'GERMANY', legendgroup = 'ABORTION'), 3, 1)
fig.add_trace(go.Bar(x = domain, y = slovakia['ABORTION'], name = 'SLOVAKIA', legendgroup = 'ABORTION'), 3, 1)
fig.add_trace(go.Bar(x = domain, y = poland['ABORTION_PER_1000'], name = 'POLAND', legendgroup = 'ABORTION_PER_1000'), 4, 1)
fig.add_trace(go.Bar(x = domain, y = germany['ABORTION_PER_1000'], name = 'GERMANY', legendgroup = 'ABORTION_PER_1000'), 4, 1)
fig.add_trace(go.Bar(x = domain, y = slovakia['ABORTION_PER_1000'], name = 'SLOVAKIA', legendgroup = 'ABORTION_PER_1000'), 4, 1)
fig.update_layout(title = 'DATA FOR POLAND, GERMANY AND SLOVAKIA BETWEEN YEARS 2000-2020')
iplot(fig)

## TAKS 5
Analiza dzietności w krajach EU na przestrzeni 2000-2020.

In [14]:
feritility_EU = data_EU_2000_2020[['YEAR', 'COUNTRY', 'FERITILITY_RANGE']]
feritility_EU.describe()

,YEAR,FERITILITY_RANGE
count,566.000000,563.000000
mean,2009.982332,1.513464
std,6.051376,0.212222
min,2000.000000,1.140000
25%,2005.000000,1.350000
50%,2010.000000,1.470000
75%,2015.000000,1.680000
max,2020.000000,2.070000


In [15]:
# prepared plot
domain = feritility_EU['YEAR']
fig = go.Figure()
fig.add_trace(go.Heatmap(x = domain,
                         y = feritility_EU['COUNTRY'],
                         z = feritility_EU['FERITILITY_RANGE']))
fig.update_layout(title = 'Współczynnik dzietności w krajach Unii Europejskiej w latach 2000-2020')
iplot(fig)

## PODSUMOWANIE
Zadanie polegało na:
1. Do listy krajów EU dodany będzie skrót trzyliterowy zgodnie z tabelą objaśnień symboli.
2. Ze wszystkich zestawień pobrancych z 'https://gateway.euro.who.int/en/' wybrane będą tylko kraje należące do EU i lata 2000 do 2020.
3. Przedstawione będą zestawienia dzietności, ilości żywych urodzeń i ilości aborcji dla każdego kraju EU jako średnia z tych 20 ostatnich lat.
4. Dla wybranych 3 krajów zaprezentowane będą statystyki dzietności, ilości żywych urodzeń i ilości aborcji dla każdego roku osobno.
5. Analiza dzietności w krajach EU na przestrzeni 2000-2020.

Z powyższych danych można odczytać, że największa ilość żywych urodzeń jest we Francji, Niemczech i Włoszech. Natomiast porównując to z współczynnikiem dzietności Francja dalej jest w czołówce, natomiast Niemcy i Włochy już nie. Wynika zatem, że liczebność tych dwóch krajów daje tak dużą ilość urodzeń, ale w przeliczeniu na pojedynczą kobietę nie jest już tak wysoka. Chwilowy wzrost dzietności nastąpił między 2007 a 2010 rokiem w części krajów UE. W większości krajów utrzymuje się on na stałym poziomie w okolicach średniej wartości.
Ciekawym jest fakt, że w krajach gdzie nie ma praktycznie aborcji, współczynnik dzietności nie są wcale większe. Pozwala to wysunąć wniosek, że albo w tych krajach ludzie lepiej się zabezpieczają przed niechcianą ciążą albo wykonują aborcję w sposób nielegalny, przez co nie ma tego w statystykach. Szczególnie widać to na zestawieniu trzech krajów Polski, Niemiec i Słowacji. Z tych trzech krajów tylko w Polsce były ograniczenia co do przeprowadzenia aborcji, a od 2015 (zmiana władzy w Polsce), współczynnik dzietności był najniższy z pośród wybranych trzech krajów.
W większości wypadków ilość aborcji wykonanych w kraju nie wpływa na dzietność. Ze statystyki wyłamuje się tylko Irlandia, gdzie współczynnik dzietności był dość wysoki przy niewielkiej ilości aborcji. Aby wysnuć jednak prawidłowe wnioski, należało by się szerzej przyjżeć zmianą jakie zaszły w Irlandii, gdzie aborcja stała się legalna dopiero w 2018 roku.